In [ ]:
'''Пример парсера coinmarketcap.com в несколько процессов (multiprocess)
Сейчас этот паресер не работает т.к. на сайте висит защита cloudflare от ddos.
Но автор говорит, что если подкрутить кое-в-каких местах, то должно прокатывать.
Взято в качестве примера по методологии самого парсинга'''

In [ ]:
'''API - легальный способ получить данные без парсинга. Так что смотри сначала,
может там есть API и выдачи с него тебе будет достаточно?
А вообще за такой парсинг могут забанить к хуям, так, что запускать этот код не рекомендую.'''

In [ ]:
'''Подход. Требуется обнаружить ссылки на главной на сами альткоины. На главной также есть пагинация и view all.
Затем перейти на страницу альткоина, получить его название и цену. ПОГНАЛИ!'''

In [ ]:
import requests
from  bs4 import BeautifulSoup
import csv
from datetime import datetime #это чтобы засекать время работы парсера над каждоым альткоином



In [ ]:
# Создаем точку входа
if __main__== '__main__':
    main()

In [ ]:
def main():
    start = datetime.now() #засекаем время начала парсинга
    url = 'https://coinmarketcap.com/alt/views/all'  #- это наша основаня ссылка
    all_links=get_all_links(get_html(url))
'''Получается такая вот матрешка.
1.Передаем url в функцию get_html. На выходе получаем станицу в txt
2.Эту страницу уже передаем в фукцию get_all_links. На выходе получаем список с ссылками'''
    for index, url in enumerate(all_links): #здесь начинается вызов функций по порядку. Очень важно не заблудится где какой параметр передается.
        html = get_html(url)
        data = get_page_data(html)
        write_csv(data)# получается, что это цикл, каскадно запускающий все эти функции работает внутри самой функции get_all_links
        print(index)#выводит порядковый номер монеты
    end =  datetime.now() #засекаем время конца парсинга
    total = end-start
    print(str(total))
    

In [ ]:
def get_html(url):
    r=requests.get(url)
    return r.text #из ответа сервера мы хотим получить текст (а точнее это будет код html-страницы в виде текста)

In [ ]:
# Этот блок исключительно для выцарапывания ссылок. Это еще на сам парсинг как таковой.
def get_all_links(html):
    soup = BeautifulSoup(html, 'lxml') # lxml - это парсер, кот. будет  использоваться
    tds = soup.find('table', id = 'currencies-all').find_all('td', _class='currency-name') #table - это имя тега, а id - его атрибут со значением
#вспоминаем. что find - это найти первое попавшееся значение, а find_all - все значения по заданному условию
    links - []
    for td in tds:
        a = td.find('a').get('href')# в объекте супа td находим тег a с атрибутом href.Это будет урл ссылки (без домена)
        domen ='https://coinmarketcap.com/' #а это сам домен с которым будет склеивать урл
        link = domen + a #конкатенация
        links.append(link) # Положили ссылочку в список links
    return links# на выходе работы функции получаем список со ссылками


In [ ]:
# А вот здесь уже начинается сам парсинг. Вытягиваем название альткоина и его цену
def get_page_data(html):
    soup = BeautifulSoup(html, 'lxml')
    try:
        name = soup.find('h1', _class='text-large').text.strip() # основная часть кода
    except:
        name='' # страховочная часть на случай исключения
    try:
        price = soup.find('span', id='quote_price').text.strip()
    except:
        price=''
    data = {'name': name, 'price': price}
    return data        

In [ ]:
# В этом блоке запишем все в csv. Заметь, что для этого нам не надо
#импортировать всю Панду - достаточно встроенного метода csv. смотри в главном модуле
def write_csv(data):
    with open('coinmarketcap.csv', 'a') as f:# а - это значит добавить в конец. Легко запомнить. т.к. это сокращение от append
        writer = csv.writer(f)
        writer.writerow( (data['name'],
                              ['price']))#тут как бы говорим, что надо записать в строку наш словарик имя-цена
#функция ничего не возвращает, а только записывает (да, так тоже можно)
        print(data['name'], 'parsed')# ну это не обязательно, так. чисто для контроля

In [ ]:
'''Это тот же самый скрипт, но он уже сделанный в многопоточном варианте '''

In [ ]:
import requests
from  bs4 import BeautifulSoup
import csv
from datetime import datetime #это чтобы засекать время работы парсера над каждым альткоином
from multiprocessing import Pool #это чтобы парсить в несколько потоков

# Создаем точку входа (она вроде как и не пригодилась нигде)
if __main__== '__main__':
    main()
    
def main():
    start = datetime.now() #засекаем время начала парсинга
    url = 'https://coinmarketcap.com/alt/views/all'  #- это наша основаня ссылка
    all_links=get_all_links(get_html(url))
'''Получается такая вот матрешка.
1.Передаем url в функцию get_html. На выходе получаем станицу в txt
2.Эту страницу уже передаем в фукцию get_all_links. На выходе получаем список с ссылками'''
    #for index, url in enumerate(all_links): #здесь начинается вызов функций по порядку. Очень важно не заблудится где какой параметр передается.
        #html = get_html(url)
        #data = get_page_data(html)
        #write_csv(data)# получается, что это цикл, каскадно запускающий все эти функции работает внутри самой функции get_all_links
        #print(index)#выводит порядковый номер монеты
    with Pool(40) as p: #40-это кол-во процессов python. По идее их столько будет в диспетчере задач
        p.map(make_all, all_links) #make_all пишется без скобок т.к. здесь не результат работы функции, а сама функция
         
    end =  datetime.now() #засекаем время конца парсинга
    total = end-start
    print(str(total))
    
def make_all(url): #Здесь сборка всех функций
        html = get_html(url)
        data = get_page_data(html)
        write_csv(data)# получается, что это цикл, каскадно запускающий все эти функции работает внутри самой функции get_all_links
    
    

def get_html(url):
    r=requests.get(url)
    return r.text #из ответа сервера мы хотим получить текст (а точнее это будет код html-страницы в виде текста)

# Этот блок исключительно для выцарапывания ссылок. Это еще на сам парсинг как таковой.
def get_all_links(html):
    soup = BeautifulSoup(html, 'lxml') # lxml - это парсер, кот. будет  использоваться
    tds = soup.find('table', id = 'currencies-all').find_all('td', _class='currency-name') #table - это имя тега, а id - его атрибут со значением
#вспоминаем. что find - это найти первое попавшееся значение, а find_all - все значения по заданному условию
    links - []
    for td in tds:
        a = td.find('a').get('href')# в объекте супа td находим тег a с атрибутом href.Это будет урл ссылки (без домена)
        domen ='https://coinmarketcap.com/' #а это сам домен с которым будет склеивать урл
        link = domen + a #конкатенация
        links.append(link) # Положили ссылочку в список links
    return links# на выходе работы функции получаем список со ссылками

# А вот здесь уже начинается сам парсинг. Вытягиваем название альткоина и его цену
def get_page_data(html):
    soup = BeautifulSoup(html, 'lxml')
    try:
        name = soup.find('h1', _class='text-large').text.strip() # основная часть кода
    except:
        name='' # страховочная часть на случай исключения
    try:
        price = soup.find('span', id='quote_price').text.strip()
    except:
        price=''
    data = {'name': name, 'price': price}
    return data

# В этом блоке запишем все в csv. Заметь, что для этого нам не надо
#импортировать всю Панду - достаточно встроенного метода csv. смотрии в главном модуле
def write_csv(data):
    with open('coinmarketcap.csv', 'a') as f:# а - это значит добавить в конец. Легко запомнить. т.к. это сокращение от append
        writer = csv.writer(f)
        writer.writerow( (data['name'],
                              ['price']))#тут как бы говорим, что надо записать в строку наш словарик имя-цена
#функция ничего не возвращает, а только записывает (да, так тоже можно)
        print(data['name'], 'parsed')# ну это не обязательно, так. чисто для контроля





In [ ]:
'''Обрати внимание, что весь парсер целиком состоит из функций, которые вызывают друг-друга '''